In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

In [ ]:
def CleaningText(TextRaw):
    
    #  1. Email Body Extraction
    if re.search(r"X-FileName:.+\n+((.+\n\n?)+)", TextRaw):
        text = re.search(r"X-FileName:.+\n+((?:.+\n\n?)+)", TextRaw).group(1)
        
        # 2. Return np.NaN, if the body begins with --Forwarded, yet it's meaningless
        if re.match(r"-{3,} Forwarded by.+\d+\/\d+\/\d+", text):
            return np.NaN
        
        # 3. If Body includes forwarding, then remove it
        if re.search(r"-{3,} Forwarded by.+\d+\/\d+\/\d+[\w\W]+", text):
            text = re.sub(r"-{3,} Forwarded by.+\d+\/\d+\/\d+[\w\W]+", "", text)
        
        # 4. If in the remaining part of the body contains original message, then discard that as well
        if re.search(r"-{5,}Original Message-{5,}[\w\W].*", text):
            text = re.sub(r"-{5,}Original Message-{5,}[\w\W]*[\n]*.*", "", text)
            
        if text:
            return text
        else:
            return np.NaN
        
    elif re.search(r"X-FileName:.+\n+(.+)", TextRaw):
        text = re.search(r"X-FileName:.+\n+(.+)", TextRaw).group(1)
        
        # 2. Return np.NaN, if the body begins with --Forwarded, yet it's meaningless
        if re.match(r"-{3,} Forwarded by.+\d+\/\d+\/\d+", text):
            return np.NaN
    
        # 3. If Body includes forwarding, then remove it
        if re.search(r"-{3,} Forwarded by.+\d+\/\d+\/\d+[\w\W]+", text):
            text = re.sub(r"-{3,} Forwarded by.+\d+\/\d+\/\d+[\w\W]+", "", text)
        
        # 4. If in the remaining part of the body contains original message, then discard that as well
        if re.search(r"-{5,}Original Message-{5,}[\w\W].*", text):
            text = re.sub(r"-{5,}Original Message-{5,}[\w\W]*[\n]*.*", "", text)
            
        if text:
            return text
        else:
            return np.NaN
    
    else:
        return np.NaN
    

In [ ]:
def EmailRead(File):
  
    with open(File, mode="r") as f:
        try:
            TextRaw = f.read()
        except UnicodeDecodeError as e:
            return None, None, None, None
    if re.search(r"^Message-ID: <(\d+\.\d+\.)JavaMail\.evans@thyme>", TextRaw):
        EmailID = re.search(r"^Message-ID: <(\d+\.\d+\.)JavaMail\.evans@thyme>", TextRaw).group(1)
    else:
        raise Exception("Message ID not found for File {}".format(File))
    if re.search(r"X-From: (.+) <?", TextRaw):
        SENDER = re.search(r"X-From: (.+) <?", TextRaw).group(1)
    else:
        print("Message ID not found for File {}".format(File))
        SENDER = np.NaN
    
    text = CleaningText(TextRaw)

    return EmailID, SENDER, text, TextRaw

In [ ]:
def ExtractEmail(AUThor):
  
    RootFolder = "./maildir/"
    Folder = ["/_sent_mail/", "/sent/", "/sent_items/"]
    DataExtract = []

    for folders in Folder:
        try:
            for message in os.listdir(RootFolder + AUThor + folders):
                if os.path.isfile(RootFolder + AUThor + folders + message):
                    EmailID, SENDER, text, TextRaw = EmailRead(RootFolder + AUThor + folders + message)
                    if EmailID != None:
                        DataExtract.append([SENDER, AUThor, message, EmailID, text, TextRaw])
        except FileNotFoundError as e:
            continue
        except NotADirectoryError as e:
            continue
    return DataExtract

In [ ]:
AUTHORS = os.listdir("./maildir")
number_author_folders = len(AUTHORS)
df = pd.DataFrame(columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text", "Email Folder"])

for AUThor in AUTHORS:
    EMAIL = ExtractEmail(author)
    if EMAIL:
        df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))

df = df.drop_duplicates(["Message ID"])
df = df[df["Text"].notna()]

# Top 20 authors are considered
print(df.value_counts(["Folder"])[:20])
df

/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2

For file ./maildir/forney-j/sent_items/158. Sender could not be found


/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2

For file ./maildir/germany-c/_sent_mail/1309. Sender could not be found
For file ./maildir/germany-c/sent_items/1026. Sender could not be found


/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))


...

/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))


For file ./maildir/lenhart-m/sent_items/235. Sender could not be found


/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2xd7y11gw_ngg0r8rx80000gn/T/ipykernel_12016/2862900019.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(emails, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
/var/folders/zc/s6g9n2

For file ./maildir/maggi-m/sent_items/22. Sender could not be found
Folder       
mann-k           8177
kaminski-v       6375
dasovich-j       4827
germany-c        4652
shackleton-s     4075
jones-t          3376
bass-e           2703
beck-s           2631
symes-k          2617
lenhart-m        2572
scott-s          2444
perlingiere-d    2339
taylor-m         2185
arnold-j         2174
love-p           2052
nemec-g          2039
fossum-d         2038
lavorato-j       1765
rogers-b         1723
delainey-d       1662
dtype: int64


,Author,Folder,File,Message ID,Text,Raw Text,Email Folder
0,John,arnold-j,36.,33491127.1075857594966.,Hey:\nHaven't had the best of months. Like yo...,Message-ID: <33491127.1075857594966.JavaMail.e...,NaN
1,John,arnold-j,667.,6384662.1075857656041.,Thank you.\n\n,Message-ID: <6384662.1075857656041.JavaMail.ev...,NaN
2,John,arnold-j,759.,21884118.1075857658063.,a couple of observations from here:\ncash/futu...,Message-ID: <21884118.1075857658063.JavaMail.e...,NaN
3,John,arnold-j,313.,11352651.1075857600972.,"Frank:\nThe $5,000,000 extra VAR disappears in...",Message-ID: <11352651.1075857600972.JavaMail.e...,NaN
4,John,arnold-j,710.,25732708.1075857656969.,don't care about the front. i think its vulne...,Message-ID: <25732708.1075857656969.JavaMail.e...,NaN
...,...,...,...,...,...,...,...
119,"Scholtes, Diana",scholtes-d,51.,14805735.1075840025496.,I received your message and those times are go...,Message-ID: <14805735.1075840025496.JavaMail.e...,NaN
120,"Scholtes, Diana",scholtes-d,18.,20865574.1075840024696.,Our Purchases:\n\n11/1\tTo PSE-EPMI MC - you h...,Message-ID: <20865574.1075840024696.JavaMail.e...,NaN
122,"Scholtes, Diana",scholtes-d,8.,16491778.1075840024450.,1\n\n,Message-ID: <16491778.1075840024450.JavaMail.e...,NaN
123,"Scholtes, Diana",scholtes-d,124.,17311779.1075840027415.,"Stewart, \n \nI left you an envelope with info...",Message-ID: <17311779.1075840027415.JavaMail.e...,NaN


In [ ]:
df.to_csv("./enron.csv")
